In [2]:
import sys
sys.path.append('..')

from src.config import SnowflakeConfig, validate_config
from src.utils.snowflake_helper import SnowflakeHelper
from src.data.dummy_data_generator import HospitalDataGenerator

# Validate configuration
validate_config()

✓ Configuration validated successfully


### Connect to Snowflake

In [3]:
config = SnowflakeConfig()
sf_helper = SnowflakeHelper(config.get_connection_params())
session = sf_helper.connect()

✓ Connected to Snowflake as harismad
  Role: "DEV_ROLE"
  Warehouse: "TEST_WAREHOUSE"
  Database: "TEST_DATABASE"
  Schema: "TEST_SCHEMA"


### Generate Dummy Data

In [6]:
sop_data = HospitalDataGenerator.generate_sop_data(20)
schedule_data = HospitalDataGenerator.generate_doctor_schedule(15)
facility_data = HospitalDataGenerator.generate_facility_data(25)

# Display samples
print("SOP Data Sample:")
print(sop_data.head(5))

print("\nDoctor Schedule Sample:")
print(schedule_data.head(5))

print("\nFacility Data Sample:")
print(facility_data.head(5))

SOP Data Sample:
     SOP_ID          SOP_CATEGORY                            SOP_TITLE  \
0  SOP-0001        Administrative     Appointment Scheduling Procedure   
1  SOP-0002       Safety Protocol  Personal Protective Equipment Usage   
2  SOP-0003          Patient Care   Medication Administration Protocol   
3  SOP-0004  Emergency Procedures           Medical Emergency Response   
4  SOP-0005          Patient Care          Patient Admission Procedure   

                                         SOP_CONTENT  DEPARTMENT  \
0  This is the detailed procedure for Appointment...  Laboratory   
1  This is the detailed procedure for Personal Pr...   Emergency   
2  This is the detailed procedure for Medication ...     Surgery   
3  This is the detailed procedure for Medical Eme...   Radiology   
4  This is the detailed procedure for Patient Adm...         ICU   

                LAST_UPDATED VERSION  
0 2025-09-22 23:43:08.269103    v3.4  
1 2025-03-09 23:43:08.269171    v3.4  
2 2025-02-05

### Load Data into Snowflake

In [7]:
sf_helper.load_data_to_table(sop_data, "hospital_sop", overwrite=True)
sf_helper.load_data_to_table(schedule_data, "doctor_schedule", overwrite=True)
sf_helper.load_data_to_table(facility_data, "hospital_facilities", overwrite=True)

print("✓ All data loaded successfully!")

✓ Loaded 20 rows to hospital_sop
✓ Loaded 45 rows to doctor_schedule
✓ Loaded 25 rows to hospital_facilities
✓ All data loaded successfully!


### Verify Data Load

In [8]:
query_sop = "SELECT COUNT(*) as count FROM hospital_sop"
query_schedule = "SELECT COUNT(*) as count FROM doctor_schedule"
query_facility = "SELECT COUNT(*) as count FROM hospital_facilities"

print(f"SOP records: {sf_helper.execute_query(query_sop)['COUNT'].iloc[0]}")
print(f"Schedule records: {sf_helper.execute_query(query_schedule)['COUNT'].iloc[0]}")
print(f"Facility records: {sf_helper.execute_query(query_facility)['COUNT'].iloc[0]}")


SOP records: 20
Schedule records: 45
Facility records: 25


### Sample Query

In [9]:
query = """
SELECT SOP_TITLE, SOP_CATEGORY, DEPARTMENT
FROM hospital_sop
WHERE SOP_CATEGORY = 'Patient Care'
LIMIT 5
"""
result = sf_helper.execute_query(query)
result

,SOP_TITLE,SOP_CATEGORY,DEPARTMENT
0,Medication Administration Protocol,Patient Care,Surgery
1,Patient Admission Procedure,Patient Care,ICU
2,Patient Transfer Guidelines,Patient Care,Radiology
3,Patient Discharge Process,Patient Care,Outpatient
4,Patient Discharge Process,Patient Care,Radiology
